# 1. Importing All the Required Libraries

In [ ]:
import tensorflow as tf
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt

# 2. Loading Data To Pipeline

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('data',batch_size=16)

In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
#get another batch from iterator
batch = data_iterator.next()

In [ ]:
#images represented as numpyarrays
batch[0].shape

- Class 0 - Drowsy driver
- Class 1 - Normal Driving 
- Class 2 - Using mobile Phone

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

# 3. Scaling Data

In [ ]:
data = data.map(lambda x,y: (x/255, y))

In [ ]:
scaled_it = data.as_numpy_iterator()

In [ ]:
batch = scaled_it.next()

In [ ]:
batch[0].max()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img)
    ax[idx].title.set_text(batch[1][idx])

# 4. Splitting Data

In [ ]:
len(data)

In [ ]:
train_size = int(len(data)*.72)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)+1

In [ ]:
train_size

In [ ]:
val_size

In [ ]:
test_size

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [ ]:
print('Test: ',len(test))
print('Validation: ',len(val))
print('Train: ',len(train))

# 5. Build Deep Learning Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.initializers import HeNormal

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3), kernel_initializer=HeNormal()))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer=HeNormal()))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer=HeNormal()))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer=HeNormal()))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer=HeNormal()))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer=HeNormal()))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer=HeNormal()))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

# 6. Train

In [ ]:
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# Saving the model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save(os.path.join('models','driver_violation.h5'))

# Testing

In [ ]:
img = cv2.imread('ca33.jpg')
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
plt.show()


In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
np.expand_dims(resize,0).shape

In [ ]:
yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
yhat

# Loading a model 

In [ ]:
img = cv2.imread('Test_Images/call.jpg')
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
np.expand_dims(resize,0).shape

In [ ]:
new_model = load_model(os.path.join('models','driver_violation.h5'))

In [ ]:
new_model.predict(np.expand_dims(resize/255, 0))

## Real time model testing 

In [ ]:
# Get a reference to webcam #0 (the default one)
video_capture = cv2.VideoCapture(0)

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Convert the frame to RGB format
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    resized_frame = tf.image.resize(frame, (256,256))

    new_model.predict(np.expand_dims(resized_frame/255, 0))

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

# Training model on new Data

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras import layers, models

In [ ]:
pretrained_model  = load_model(os.path.join('models','driver_violation.h5'))

In [ ]:
# To freeze some layers of the pre-trained model to prevent them from being updated during fine-tuning
for layer in pretrained_model.layers:
    layer.trainable = False

In [ ]:
# Remove the last layer
pretrained_model.pop()

In [ ]:
new_model = models.Sequential([
    pretrained_model,
    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_initializer='he_normal'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')  # Adjust num_classes based on your new dataset
])

In [ ]:
new_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
new_model.summary()

In [ ]:
new_data = tf.keras.utils.image_dataset_from_directory('new_data',batch_size=16)

In [ ]:
data_iterator = new_data.as_numpy_iterator()
batch = data_iterator.next()
batch[0].shape

In [ ]:
new_data = new_data.map(lambda x,y: (x/255, y))

In [ ]:
scaled_it = new_data.as_numpy_iterator()
batch = scaled_it.next()
batch[0].max()

In [ ]:
len(new_data)

In [ ]:
train_size = int(len(new_data)*.72)+1
val_size = int(len(new_data)*.3)

In [ ]:
train_size

In [ ]:
val_size

In [ ]:
train = new_data.take(train_size)
val = new_data.skip(train_size).take(val_size)

In [ ]:
new_model.fit(train, epochs=100, validation_data=val)

In [ ]:
new_model.save(os.path.join('models','driver_violation_v2.h5'))

In [ ]:
img = cv2.imread('captured_images/captured_image_1709974888.jpg')
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
plt.show()


In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
np.expand_dims(resize,0).shape

In [ ]:
yhat = new_model.predict(np.expand_dims(resize/255, 0))

In [ ]:
yhat